<a href="https://colab.research.google.com/github/natmonge/UCDPA_NataliaMongelos/blob/main/Data_Analytics_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# packages

# standard
import numpy as np
import pandas as pd
import requests
import urllib.parse

# plots
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px



# IMPORTING 

In [ ]:
# wrapper to API call

def get_api_call(ids, **kwargs):
    API_BASE_URL = "https://apis.datos.gob.ar/series/api/"
    kwargs["ids"] = ",".join(ids)
    return "{}{}?{}".format(API_BASE_URL, "series", urllib.parse.urlencode(kwargs))
    
# convert the API call in CSV to a DataFrame using pandas
df = pd.read_csv(get_api_call(
    ["380.3_ICC_NACIONNAL_0_T_12"],
    format="csv", start_date=2019
))

print (df)